# Práctica dirigida: Fundamentos de los AE


In [1]:
import math
import random


### 0.1 Representación de un decimal en binario

In [2]:

def decToBin(dec:int, len = 0):
    a = dec
    result = ""
    if a == 0: result = "0"
    while(a > 0):
        result = str(a&1) +result
        a = a >> 1

    if len != 0: result = result.zfill(len) 
    return  result




print(decToBin(0 ))
print(decToBin(4 ))
print(decToBin(0b1011000))
print(decToBin(0b1111111))
print(decToBin(78))#	1001110
print(decToBin(79))#	1001111



0
100
1011000
1111111
1001110
1001111


In [3]:
def binToGray (a : int):
    shiftedA =  a >> 1
    return a ^ shiftedA

def grayToBin (gray : int):
    grayBits= gray
    result  = gray
    while(grayBits > 0 ):
        grayBits =grayBits >> 1
        result = result ^ grayBits
    return result


print(decToBin(binToGray(0b0000)))
print(decToBin(binToGray(0b0001)))
print(decToBin(binToGray(0b0010)))
print(decToBin(binToGray(0b0011)))
print(decToBin(binToGray(0b0100)))

print(decToBin(grayToBin(0b00000)))
print(decToBin(grayToBin(0b00001)))
print(decToBin(grayToBin(0b00011)))
print(decToBin(grayToBin(0b00010)))
print(decToBin(grayToBin(0b00110)))



0
1
11
10
110
0
1
10
11
100


### 0.2 Generar individuos

In [19]:
class Domain:
    def __init__(self, min, max, step):
        self.min = min
        self.max = max
        self.step = step
        self.len = round((max - min) / step) + 1
        self.maxBin = self.toBinScale(max)
        self.genSize = len(decToBin(self.maxBin))
        

    def toBinScale( self , n  ):
        return round((n - self.min) / self.step)

    def toDomScale( self , n  ):
        return int(n * self.step + self.min)
    
    def toBinaryList(self , a :list):
        return [decToBin(x, self.genSize) for x in a] 

    def toValuesList(self , a :list):
        return [self.toDomScale(x) for x in a] 



def generateAll(min , max , step , generateGray = False ):
    result = []
    domain =  Domain(min ,  max  , step)
    for i in range (0, domain.maxBin +1):
        result.append(i)
    
    resultBin = [binToGray(x) for x in result] if generateGray else result

    return domain.toBinaryList(resultBin),  domain.toValuesList(result)


print(generateAll(-5 , 5 , 1 ))

print(generateAll(-5 , 5 , 1 , generateGray= True))


(['0000', '0001', '0010', '0011', '0100', '0101', '0110', '0111', '1000', '1001', '1010'], [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5])
(['0000', '0001', '0011', '0010', '0110', '0111', '0101', '0100', '1100', '1101', '1111'], [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5])


In [20]:
def generateRandom(number ,domain:Domain , generateGray=False ):
    selection = random.sample(range(0, domain.maxBin), number)

    selectionBin = [binToGray(x) for x  in selection] if generateGray else selection

    return domain.toBinaryList(selectionBin) , domain.toValuesList(selection)


In [23]:
print(generateRandom ( 10,Domain (-5 , 5 , 1 ))) 


(['0001', '0100', '0110', '1000', '0111', '0101', '0011', '1001', '0010', '0000'], [-4, -1, 1, 3, 2, 0, -2, 4, -3, -5])


In [24]:
print(generateRandom ( 10,Domain (-5 , 5 , 1 ) , generateGray=True)) 


(['0101', '0110', '0001', '0000', '0011', '1100', '1101', '0100', '0010', '0111'], [1, -1, -4, -5, -3, 3, 4, 2, -2, 0])


In [27]:

#x ∈ [−5, 5] binario
print(generateRandom ( 10,Domain (-5 , 5 , 1 ) )) #Genera en forma binaria standard
#x ∈ [−5, 5] gray code
print(generateRandom ( 10,Domain (-5 , 5 , 1 ) , generateGray=True)) 

#y ∈ [0, 10] binario
print(generateRandom ( 10,Domain (0 , 10 , 1 ) )) 
#y ∈ [0, 10] grey code
print(generateRandom ( 10,Domain (0 , 10 , 1 ) , generateGray=True)) 


(['0110', '0001', '0101', '0000', '0010', '0100', '1000', '1001', '0011', '0111'], [1, -4, 0, -5, -3, -1, 3, 4, -2, 2])
(['0000', '0001', '0010', '0100', '1100', '0011', '1101', '0111', '0110', '0101'], [-5, -4, -2, 2, 3, -3, 4, 0, -1, 1])
(['0011', '0001', '0111', '0101', '1000', '0100', '0110', '0010', '0000', '1001'], [3, 1, 7, 5, 8, 4, 6, 2, 0, 9])
(['0011', '0101', '0001', '0110', '0000', '1101', '0111', '1100', '0100', '0010'], [2, 6, 1, 4, 0, 9, 5, 8, 7, 3])


###  0.3 Construyendo los operadores básicos

In [57]:
class Population :
    def __init__(self, domain:Domain , size:int , usingGrayCode = False):
        self.domain = domain
        self.size = size
        self.usingGrayCode = usingGrayCode
        self.population= self.initialPopulation()
    
    def populationValues(self):
        return [self.domain.toDomScale(int(x , 2)) for x in self.population]

    def  initialPopulation(self):
        return generateRandom(self.size ,self.domain , self.usingGrayCode )[0]
        
    def reconbination(self , gena:str , genb :str ):
        point  =random.randint(1,self.domain.genSize -1) 
        return  gena [:point] + genb [point:] 
    
    def seleccion(self , size):
        self.population = random.sample(self.population, size)

    def mutation (self , gen :str ):
        point  =random.randint(1,self.domain.genSize -1) 
        return gen [:point] + ("1" if gen[point] == "0" else "0") + gen [point+1:]


population55 = Population (Domain(-5 , 5 , 1) , 4)
print(population55.population)


print(population55.reconbination("1111" , "0000" ))

population55.seleccion(2)
print(population55.population)


print(population55.mutation("1111"))


['0011', '1001', '0100', '0010']
1100
['0011', '1001']
1110


In [58]:
def optimize (pop:Population , iterations:int , f):
    # Algoritmo canonico
    for i in range(iterations):
        fitness =  [f(x) for x in pop.populationValues()]
        print(fitness , pop.population)


toOptimize = lambda x : bin(x).count("1")
optimize(Population (Domain(0 , 16 , 1) , 4), 1, toOptimize)



[1, 2, 2, 3] ['01000', '01100', '00101', '01011']


In [50]:
[x for x in zip([1,1,1] ,[2,3,4])]

[(1, 2), (1, 3), (1, 4)]